# Divide and Conquer Algorithms

**Contents:**
1. Divide and conquer philosophy
2. Counting Inversions problem (Collaborative filtering problem) using brute force vs divide and conquer
3. Strassens's Matrix Multiplication Algorithm
4. Closest Pair Algorithm (for nlogn time)

**What are the steps in divide and conquer?**
1. Divide each problem into smaller subproblems
2. Solve smaller subproblems recursively
3. Merge the smaller subproblems together

The most ingenuity is needed for 3. The merge step. The recursive calls can be done in the return statement for simple problems (like fibonnaci, factorials etc) but for 
larger problems we may need the 1. Division step to be done by calling itself in the middle and the 3. The merge step done in another helper function (like in mergesort shown in the "1_introduction.ipynb"). The merge step is sometimes of linear complexity which is why we see nlog(n) complexity so often, the log(n) from the division step and the linear complexity from the merge.

We can use recursive trees to analyse time complexity of such recursive divide and conquer algorithms. For example in mergesort, as we keep halving the size of the array, we have log(n) levels to our recursive tree. Then we analyse the balance between number of recursive calls used in a subproblem (increasing our time complexity) vs the amount of work needed to be done by our subproblem (the more we divide our problem the more we decrease our time complexity). In mergesort, the function calls itself twice, so at each level we have 2^j subproblems underneath its level in the tree. The complexity of our merge function is linear, so it has complexity n, but as the size of this subproblem halves, so n/(2^j). Now we multiply the two 2^j * n/(2^j) = n. Times this by the depth of our tree (log(n)) to get nlogn.

# Counting Inversions

AKA collaborative filtering algorithm for determining similarity of preferences. For example you and a friend rank movies from 1-10. Then create an array where a[1] contains your friends ranking of your favourite movie, a[2] contains their ranking of your second favourite etc. If both your rankings are the same, the list created would be sorted. The more "inversions" needed to sort the list tells how disimilar your preferences are.

**Brute force approach**

In [16]:
# Given array input arr, how many inversions do you need?

def inversion_brute(arr):
    count_inv = 0
    for i in range(1, len(arr)-1):
        for j in range(i+1, len(arr)):
            if arr[i] > arr[j]:
                count_inv += 1

    return count_inv

In [17]:
print(inversion_brute([1,3,5,2,4,6]))

3


**Divide and Conquer approach**

In [41]:
def inversion_count(arr):

    if len(arr) <= 1:
        return (arr, 0)
    
    C, left_side_invt = inversion_count(arr[:len(arr)//2])
    D, right_side_invt = inversion_count(arr[len(arr)//2:])

    B, split_invt = inversion_merge(C, D, arr)

    return (B, left_side_invt + right_side_invt + split_invt)

def inversion_merge(arr_l, arr_r, arr):
    i = j = k = 0
    count_inv = 0

    while i < len(arr_l) and j < len(arr_r):
        if arr_l[i] < arr_r[j]:
            arr[k] = arr_l[i]
            i += 1
        else:
            arr[k] = arr_r[j]

            # If item from arr_r copied into arr before item in arr_l,
            # the the number of inverses needs to be incremented by the
            # number of items remaining in arr_l at this point
            count_inv += len(arr_l) - i


            j += 1
        k += 1

    while i < len(arr_l):
        arr[k] = arr_l[i]
        i += 1
        k += 1
    
    while j < len(arr_r):
        arr[k] = arr_r[j]
        j += 1
        k += 1

    return arr, count_inv



In [45]:
print(inversion_count([1,3,5,2,4,6]))
print(inversion_count([6,5,4,3,2,1]))

([1, 2, 3, 4, 5, 6], 3)
([1, 2, 3, 4, 5, 6], 15)


# Strassen's Matrix Multiplication Algorithm

Complexity of normal matrix addition/ subtraction methods are n^2 complexity. Matrix mutliplication using its definition is n^3 complexity.
We can develop a recursive matrix multiplication method, but this will also be n^3 complexity.

*However* by using the recursive method we open the doors to possible reduce the number of recursive calls (lowering the number of calls just by 1 is enough to have massive gains as this is compounded on every call to itself, leading to an order less complexity often times). Strassen's algorithm did just this, firstly implement the normal recursive method of matrix multiplication which works splitting each matrix into quarters (a,b,c,d and e,f,g,h) and then making 8 recursive matmul calls (ae, bg, af, bh, ce, dg, cf, dh) and then changing this in a clever way which reduces the number of recursive calls to 7 instead of 8.

This improvement means a 12.5% performance increase in every call of the function, and when this is compounded, it leads to the overall algorithm having nlogn complexity

# Iterative Matrix Multiplication Algorithm

In [59]:
# Firstly, we implement matmul by its iterative definition.

def matmul(x,y):
    x_col = len(x[0])
    x_row = len(x)
    y_col = len(y[0])
    y_row = len(y)
    if x_col is not y_row:
        raise TypeError("These two matrices cannot be multiplied!")
    
    z = [[0 for i in range(y_col)] for j in range(x_row)]

    for i in range(x_row):
        for j in range(y_col):
            # z_ij = dot product of ith row of x and jth column of y
            z[i][j] = dot(x,y,i,j)
    
    return z

def dot(x,y,i,j):
    total = 0
    i = x[i]
    j = [y[k][j] for k in range(len(y))]

    return sum(p*q for p,q in zip(i,j))


In [60]:
matmul([[1,2,3],[4,5,6],[7,8,9]],[[1,2],[3,4],[5,6]])

[[22, 28], [49, 64], [76, 100]]

# Recursive Matrix Multiplication Algorithm

In [115]:
import numpy as np
def rmatmul(x,y):
    # Lets assume we are doing 2 square matrices of equal dimensions where num_rows = num_columns = power of 2

    if len(x) <= 1:
        return x[0][0]*y[0][0]

    # if x was a numpy array we could simply do x[:len(x)//2,:len(x[0])//2]
    a = [i[:len(x[0])//2] for i in x[:len(x)//2]]
    c = [i[:len(x[0])//2] for i in x[len(x)//2:]]
    b = [i[len(x[0])//2:] for i in x[:len(x)//2]]
    d = [i[len(x[0])//2:] for i in x[len(x)//2:]]

    e = [i[:len(y[0])//2] for i in y[:len(y)//2]]
    g = [i[:len(y[0])//2] for i in y[len(y)//2:]]
    f = [i[len(y[0])//2:] for i in y[:len(y)//2]]
    h = [i[len(y[0])//2:] for i in y[len(y)//2:]]

    # 8 Recursive calls
    ae = rmatmul(a,e)
    bg = rmatmul(b,g)
    af = rmatmul(a,f)
    bh = rmatmul(b,h)
    ce = rmatmul(c,e)
    dg = rmatmul(d,g)
    cf = rmatmul(c,f)
    dh = rmatmul(d,h)

    return [[mat_add(ae,bg), mat_add(af,bh)],[mat_add(ce,dg), mat_add(cf,dh)]]

def mat_add(x,y):
    if type(x) == int:
        return x+y
    
    z = [[0 for i in range(len(x[0]))] for j in range(len(x))]

    for i in range(len(x)):
        for j in range(len(x[0])):
            z[i][j] = x[i][j] + y[i][j]


    return z



In [108]:
matmul([[1,2,3,9],[4,5,6,9],[7,8,9,9],[10,11,12,9]],[[13,14,15,16],[17,18,19,20],[21,22,23,24],[25,26,27,28]])

[[335, 350, 365, 380],
 [488, 512, 536, 560],
 [641, 674, 707, 740],
 [794, 836, 878, 920]]

In [117]:
rmatmul([[1,2,3,9],[4,5,6,9],[7,8,9,9],[10,11,12,9]],[[13,14,15,16],[17,18,19,20],[21,22,23,24],[25,26,27,28]])

# Please note that this still displays the results as [[a],[b], [c], [d]] where a,b,c and d are all 2x2 matrices, so although the 
# formatting looks different the result is the same

[[[[335, 350], [488, 512]], [[365, 380], [536, 560]]],
 [[[641, 674], [794, 836]], [[707, 740], [878, 920]]]]

# Strassen Matrix Multiplication Algorithm

Strassen found a way to save a recursive call

P1 = A · (F - H)

P2 = (A + B) · H

P3 = (C + D) · E

P4 = D · (G - E)

P5 = (A + D) · (E + H)

P6 = (B - D) · (G + H)

P7 = (A - C) · (E + F)

x*y = [[(p5 + p4 - p2 + p6), (p1 + p2)],[(p3 + p4), (p1 + p5 - p3 -p7)]]


In [124]:
import numpy as np


def strassmatmul(x, y):
    # Lets assume we are doing 2 square matrices of equal dimensions where num_rows = num_columns = power of 2

    if len(x) <= 1:
        return x[0][0]*y[0][0]

    # if x was a numpy array we could simply do x[:len(x)//2,:len(x[0])//2]
    a = [i[:len(x[0])//2] for i in x[:len(x)//2]]
    c = [i[:len(x[0])//2] for i in x[len(x)//2:]]
    b = [i[len(x[0])//2:] for i in x[:len(x)//2]]
    d = [i[len(x[0])//2:] for i in x[len(x)//2:]]

    e = [i[:len(y[0])//2] for i in y[:len(y)//2]]
    g = [i[:len(y[0])//2] for i in y[len(y)//2:]]
    f = [i[len(y[0])//2:] for i in y[:len(y)//2]]
    h = [i[len(y[0])//2:] for i in y[len(y)//2:]]

    # 8 Recursive calls

    p1 = strassmatmul(a, matt_op(f,h, sub))
    p2 = strassmatmul(matt_op(a,b, add), h)
    p3 = strassmatmul(matt_op(c,d, add),e)
    p4 = strassmatmul(d, matt_op(g,e,sub))
    p5 = strassmatmul(matt_op(a,d, add),matt_op(e,h,add))
    p6 = strassmatmul(matt_op(b,d, sub), matt_op(g,h,add))
    p7 = strassmatmul(matt_op(a,c,sub),matt_op(e,f,add))

    return [[matt_op(matt_op(matt_op(p5, p4, add), p2, sub), p6, add), matt_op(p1, p2, add)], [matt_op(p3, p4, add), matt_op(matt_op(matt_op(p1, p5, add), p3, sub), p7, sub)]]


def matt_op(x, y, func):
    if type(x) == int:
        return func(x,y)

    z = [[0 for i in range(len(x[0]))] for j in range(len(x))]

    for i in range(len(x)):
        for j in range(len(x[0])):
            z[i][j] = func(x[i][j], y[i][j])

    return z

def add(x,y):
    return x+y

def sub(x,y):
    return x-y


In [125]:
strassmatmul([[1, 2, 3, 9], [4, 5, 6, 9], [7, 8, 9, 9], [10, 11, 12, 9]], [
        [13, 14, 15, 16], [17, 18, 19, 20], [21, 22, 23, 24], [25, 26, 27, 28]])

# Please note that this still displays the results as [[a],[b], [c], [d]] where a,b,c and d are all 2x2 matrices, so although the
# formatting looks different the result is the same


[[[[335, 350], [488, 512]], [[365, 380], [536, 560]]],
 [[[641, 674], [794, 836]], [[707, 740], [878, 920]]]]

# The Closest Pair Problem